In [17]:
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine

In [36]:
pg_url = 'postgresql://root:root@pg_database:5432/ny_taxi'

def chunker(df, size):
    # from http://stackoverflow.com/a/434328
    return (df[pos:pos + size] for pos in range(0, len(df), size))

def insert_with_progress(df, table_name, con, mode="replace"):
    chunk_size = int(len(df) / 100)
    with tqdm(total=len(df)) as progress_bar:
        for i, _df in enumerate(chunker(df, chunk_size)):
            _df.to_sql(con=con, name=table_name, if_exists=mode, index=False)
            progress_bar.update(chunk_size)


def load_file(file_path: str, table_name: str, pg_url: str) -> None:
    if file_path.endswith('.parquet'):
        df = pd.read_parquet(file_path)
    elif file_path.endswith('.csv') or file_path.endswith('.csv.gz'):
        df = pd.read_csv(file_path)
    else:
        return
    df.columns = [c.lower() for c in df.columns] # postgres doesn't like capitals or spaces
    engine = create_engine(pg_url)
    insert_with_progress(df, table_name, engine)

In [31]:
create_engine(pg_url).connect()

In [32]:
!ls data

data_dictionary_trip_records_yellow.pdf  yellow_tripdata_2021-01.csv.gz
green_tripdata_2019_01.csv.gz		 yellow_tripdata_2021_01.parquet
taxi_zone_lookup.csv


In [33]:
load_file('data/yellow_tripdata_2021_01.parquet', 'yellow_tripdata_2021_01', pg_url)

1383397it [02:00, 11442.51it/s]                             


In [34]:
load_file('data/taxi_zone_lookup.csv', 'taxi_zone_lookup', pg_url)

266it [00:04, 62.00it/s]                         


In [35]:
load_file('data/green_tripdata_2019_01.csv.gz', 'green_tripdata_2019_01', pg_url)

637209it [01:01, 10288.13it/s]                            
